The goal of this analysis is to try and refine the 3' end calling of TALON by using the GENCODE strategy of longest TES per transcript.

In [2]:
import pandas as pd
import numpy as np
import csv

In [3]:
def get_longest_ends(annot, how='tes', novelty='novel'):
    df = pd.read_csv(annot, sep='\t')
    
    if novelty == 'novel':
        df = df.loc[df.transcript_novelty != 'Known']
    
    fwd = df.loc[df.strand == '+']
    rev = df.loc[df.strand == '-']
    
    # furthest downstream for tes
    # if + strand, max coord of read end
    # if - strand, min coord of read end
    if how == 'tes':
        fwd = fwd[['transcript_ID', 'read_end']]
        fwd = fwd.groupby('transcript_ID').max().reset_index()
        rev = rev[['transcript_ID', 'read_end']]
        rev = rev.groupby('transcript_ID').min().reset_index()
        
        
    # furthest upstream for tss:
    # if + strand, min coord of read start
    # if - strand, max coord of read start
    elif how == 'tss':
        fwd = fwd[['transcript_ID', 'read_start']]
        fwd = fwd.groupby('transcript_ID').min().reset_index()
        rev = rev[['transcript_ID', 'read_start']]
        rev = rev.groupby('transcript_ID').max().reset_index()
    
    # concat fwd and rev
    df = pd.concat([fwd, rev])
    
    return df

In [23]:
# get the longest ends from the read annotation file
# annot: TALON read annotation file path
# how: 'tss' or 'tes', tss will find start ends and tes will find stop ends
# gtf: gtf file location
# ends: df with transcript_ID, end coordinate
# how: 'tss' or 'tes'
# opref: output file prefix
def replace_gtf_end_coords(gtf, ends, opref, how='tes', test=False, verbose=False):

    # read preexisting GTF and
    gtf_df = pd.read_csv(gtf, sep='\t', header=None, \
                names=['chr', 'source', 'entry_type', \
                       'start', 'stop', 'score', 'strand',\
                       'frame', 'fields'], comment='#')

    # get relevant values from fields
    gtf_df['transcript_id'] = np.nan
    gtf_df.loc[gtf_df.entry_type!='gene', 'transcript_id'] = gtf_df.loc[gtf_df.entry_type!='gene'].fields.str.split(pat='talon_transcript "', n=1, expand=True)[1]
    gtf_df.loc[gtf_df.entry_type!='gene', 'transcript_id'] = gtf_df.loc[gtf_df.entry_type!='gene'].transcript_id.str.split(pat='"', n=1, expand=True)[0]


    if how == 'tes':
        ends.columns = ['transcript_id', 'tes']
    elif how == 'tss':
        ends.columns = ['transcript_id', 'tss']

    # merge gtf_df with end information
#     ends.transcript_id = ends.transcript_id.astype('str')
    df = gtf_df.loc[gtf_df.transcript_id.notnull()]
    ends.transcript_id = ends.transcript_id.astype('str')
    gtf_df.transcript_id = gtf_df.transcript_id.astype('str')
    gtf_df = gtf_df.merge(ends, how='left', on='transcript_id')
    df.transcript_id = df.transcript_id.astype('str')
    df = df.merge(ends, how='inner')

    if test:
        print('Before editing')
        print(gtf_df[['transcript_id', 'entry_type', 'strand', 'start', 'stop', how]])

    # swap out read starts or ends for the longest ones
    tids = df.transcript_id.unique()
    for t, tid in enumerate(tids):
        if t % 1000 == 0 and verbose:
            print('Processing transcript {} of {}'.format(t, len(tids)))

        # fwd: swap out transcript "stop" and last exon "stop"
        # rev: swap out transcript "start" and last exon "start"
        if how == 'tes':
            # tes fwd
            ind = gtf_df.loc[(gtf_df.strand=='+')&(gtf_df.transcript_id==tid)].index.tolist()
            if ind:
                # stop of transcript for fwd
                i = ind[0]
                gtf_df.loc[i, 'stop'] = gtf_df.loc[i, 'tes']
                # stop of last exon for fwd
                i = ind[-1]
                gtf_df.loc[i, 'stop'] = gtf_df.loc[i, 'tes']

            # tes rev
            ind = gtf_df.loc[(gtf_df.strand=='-')&(gtf_df.transcript_id==tid)].index.tolist()
            if ind:
                # start of trancscript for rev
                i = ind[0]
                gtf_df.loc[i, 'start'] = gtf_df.loc[i, 'tes']
                # start of last exon for rev
                i = ind[-1]
                gtf_df.loc[i, 'start'] = gtf_df.loc[i, 'tes']
        # fwd: swap out transcript "start" and first exon "start"
        # rev: swap out transcript "stop" and first exon "stop"
        elif how == 'tss':
            # tss fwd
            ind = gtf_df.loc[(gtf_df.strand=='+')&(gtf_df.transcript_id==tid)].index.tolist()
            if ind:
                # start of transcript for fwd
                i = ind[0]
                gtf_df.loc[i, 'start'] = gtf_df.loc[i, 'tss']
                # start of first exon for fwd
                i = ind[1]
                gtf_df.loc[i, 'start'] = gtf_df.loc[i, 'tss']
            # tss rev
            ind = gtf_df.loc[(gtf_df.strand=='-')&(gtf_df.transcript_id==tid)].index.tolist()
            if ind:
                # stop of transcript for rev
                i = ind[0]
                gtf_df.loc[i, 'stop'] = gtf_df.loc[i, 'tss']
                # stop of first exon for rev
                i = ind[1]
                gtf_df.loc[i, 'stop'] = gtf_df.loc[i, 'tss']
    
    # now fix gene coordinates
    gtf_df['gene_id'] = gtf_df.loc[gtf_df.entry_type!='gene'].fields.str.split(pat='talon_gene "', n=1, expand=True)[1]
    gtf_df['gene_id'] = gtf_df.loc[gtf_df.entry_type!='gene'].gene_id.str.split(pat='"', n=1, expand=True)[0]

    # tes
    if how == 'tes':
        print('hewwo?')
        # fwd: replace "stop" of the gene with the maximum of the "stops"
        gene_ind = gtf_df.loc[(gtf_df.strand == '+')&(gtf_df.entry_type=='gene')&(gtf_df.tes.notnull())].index.tolist()
        if gene_ind:
            fwd = gtf_df.loc[(gtf_df.strand == '+')&(gtf_df.entry_type=='transcript')]
            if test:
                print('fwd')
                print(gtf_df.loc[gene_ind])
            gtf_df.loc[gene_ind, 'stop'] = gtf_df.loc[gene_ind].apply(lambda x: \
                fwd.loc[fwd.gene_id==x.gene_id, 'stop'].max(), axis=1)
        
        # rev: replace "start" of the gene with the minimum of the "starts"
        gene_ind = gtf_df.loc[(gtf_df.strand == '-')&(gtf_df.entry_type=='gene')&(gtf_df.tes.notnull())].index.tolist()
        if gene_ind:
            rev = gtf_df.loc[(gtf_df.strand == '-')&(gtf_df.entry_type=='transcript')]
            if test: 
                print('rev')
                print(gtf_df.loc[gene_ind])
            gtf_df.loc[gene_ind, 'start'] = gtf_df.loc[gene_ind].apply(lambda x: \
                rev.loc[rev.gene_id==x.gene_id, 'start'].min(), axis=1)
        
    # tss
    elif how == 'tss':
        # fwd: replace "start" of the gene with the minimum of the "starts"
        gene_ind = gtf_df.loc[(gtf_df.strand == '+')&(gtf_df.entry_type=='gene')&(gtf_df.tss.notnull())].index.tolist()
        if gene_ind: 
            fwd = gtf_df.loc[(gtf_df.strand == '+')&(gtf_df.entry_type=='transcript')]
            gtf_df.loc[gene_ind, 'start'] = gtf_df.loc[gene_ind].apply(lambda x: \
                fwd.loc[fwd.gene_id==x.gene_id, 'start'].min(), axis=1)
        
        # rev: replace "stop" of the gene with the maximum of the "stops"
        gene_ind = gtf_df.loc[(gtf_df.strand == '-')&(gtf_df.entry_type=='gene')&(gtf_df.tss.notnull())].index.tolist()
        if gene_ind:
            rev = gtf_df.loc[(gtf_df.strand == '-')&(gtf_df.entry_type=='transcript')]
            gtf_df.loc[gene_ind, 'stop'] = gtf_df.loc[gene_ind].apply(lambda x: \
                rev.loc[rev.gene_ind==x.gene_ind, 'stop'].max(), axis=1)

    if test:
        print()
        print('After editing')
        print(gtf_df[['transcript_id', 'entry_type', 'strand', 'start', 'stop', how]])

    cols=['chr', 'source', 'entry_type', \
          'start', 'stop', 'score', 'strand',\
           'frame', 'fields']
    gtf_df = gtf_df[cols]
    gtf_df['start'] = gtf_df['start'].astype('int')
    gtf_df['stop'] = gtf_df['stop'].astype('int')
    if test:
        fname = '{}_revised_{}_test.gtf'.format(opref, how)
    else:
        fname = '{}_revised_{}.gtf'.format(opref, how)
    gtf_df.to_csv(fname, sep='\t', header=None, index=False, quoting=csv.QUOTE_NONE)
    return gtf_df, fname

#### test with 4 transcripts

In [24]:
annot = 'PacBio_Brain_talon_read_annot.tsv'
gtf = 'test.gtf'

tes = get_longest_ends(annot, how='tes', novelty='novel')
df = replace_gtf_end_coords(gtf, tes, 'beep', how='tes', test=True)

Before editing
   transcript_id  entry_type strand     start      stop         tes
0            nan        gene      -   3206523   3215632         NaN
1              4  transcript      -   3206523   3215632         NaN
2              4        exon      -   3213439   3215632         NaN
3              4        exon      -   3206523   3207317         NaN
4            nan        gene      +   4857814   4897909         NaN
5             63  transcript      +   4857814   4897905         NaN
6             63        exon      +   4857814   4857976         NaN
7             63        exon      +   4867470   4867532         NaN
8             63        exon      +   4878027   4878132         NaN
9             63        exon      +   4886744   4886831         NaN
10            63        exon      +   4889457   4889602         NaN
11            63        exon      +   4890740   4890796         NaN
12            63        exon      +   4891915   4892069         NaN
13            63        exon     

In [53]:
df = replace_gtf_end_coords(gtf, tes, 'beep', how='tes', test=True)

Before editing
   transcript_id  entry_type strand    start     stop        tes
0            nan        gene      -  3206523  3215632        NaN
1              4  transcript      -  3206523  3215632        NaN
2              4        exon      -  3213439  3215632        NaN
3              4        exon      -  3206523  3207317        NaN
4            nan        gene      +  4857814  4897905        NaN
5         142372  transcript      +  4857814  4897905  4899923.0
6         142372        exon      +  4857814  4857976  4899923.0
7         142372        exon      +  4867470  4867532  4899923.0
8         142372        exon      +  4878027  4878132  4899923.0
9         142372        exon      +  4886744  4886831  4899923.0
10        142372        exon      +  4889460  4889602  4899923.0
11        142372        exon      +  4890740  4890796  4899923.0
12        142372        exon      +  4891915  4892069  4899923.0
13        142372        exon      +  4893417  4893563  4899923.0
14        

In [36]:
df.head()

,chr,source,entry_type,start,stop,score,strand,frame,fields
0,chr1,HAVANA,gene,3206523.0,3215632,.,-,.,"gene_id ""ENSMUSG00000051951.5""; gene_name ""Xkr..."
1,chr1,HAVANA,transcript,NaN,3215632,.,-,.,"gene_id ""ENSMUSG00000051951.5""; transcript_id ..."
2,chr1,HAVANA,exon,3213439.0,3215632,.,-,.,"gene_id ""ENSMUSG00000051951.5""; transcript_id ..."
3,chr1,HAVANA,exon,3206523.0,3207317,.,-,.,"gene_id ""ENSMUSG00000051951.5""; transcript_id ..."
4,chr1,HAVANA,gene,4857814.0,4897905,.,+,.,"gene_id ""ENSMUSG00000033813.15""; gene_name ""Tc..."


In [19]:
# annot = 'PacBio_Brain_talon_read_annot.tsv'
# gtf = 'Brain_talon.gtf'

# tes = get_longest_ends(annot, how='tes', novelty='novel')
# df = replace_gtf_end_coords(gtf, tes, 'beep', how='tes', test=True)

In [1]:
# cortex
annot = 'PacBio_Brain_talon_read_annot.tsv'
gtf = 'mouse_brain_cortex_talon.gtf'

tes = get_longest_ends(annot, how='tes', novelty='novel')

NameError: name 'get_longest_ends' is not defined

In [55]:
cort_df = replace_gtf_end_coords(gtf, tes, 'mouse_brain_cortex', how='tes')

Processing transcript 0 of 5865
Processing transcript 1000 of 5865
Processing transcript 2000 of 5865
Processing transcript 3000 of 5865
Processing transcript 4000 of 5865
Processing transcript 5000 of 5865


In [56]:
# hippocampus
annot = 'PacBio_Brain_talon_read_annot.tsv'
gtf = 'mouse_brain_hippocampus_talon.gtf'

tes = get_longest_ends(annot, how='tes', novelty='novel')

In [57]:
hipp_df = replace_gtf_end_coords(gtf, tes, 'mouse_brain_hippocampus', how='tes')

Processing transcript 0 of 5865
Processing transcript 1000 of 5865
Processing transcript 2000 of 5865
Processing transcript 3000 of 5865
Processing transcript 4000 of 5865
Processing transcript 5000 of 5865


In [48]:
# REEEEEEFROAMT
gtf = 'mouse_brain_cortex_revised_tes.gtf'
gtf_df = pd.read_csv(gtf, sep='\t', header=None, \
                names=['chr', 'source', 'entry_type', \
                       'start', 'stop', 'score', 'strand',\
                       'frame', 'fields'], comment='#')
gtf_df['start'] = gtf_df.start.astype('int')
gtf_df['stop'] = gtf_df.stop.astype('int')
fname = 'cortex_revised_tes.gtf'
gtf_df.to_csv(fname, sep='\t', header=None, index=False, quoting=csv.QUOTE_NONE)



ValueError: Cannot convert non-finite values (NA or inf) to integer

In [49]:
gtf_df.head()


,chr,source,entry_type,start,stop,score,strand,frame,fields
0,chr1,HAVANA,gene,3206523.0,3215632.0,.,-,.,"gene_id ""ENSMUSG00000051951.5""; gene_name ""Xkr..."
1,chr1,HAVANA,transcript,NaN,3215632.0,.,-,.,"gene_id ""ENSMUSG00000051951.5""; transcript_id ..."
2,chr1,HAVANA,exon,3213439.0,3215632.0,.,-,.,"gene_id ""ENSMUSG00000051951.5""; transcript_id ..."
3,chr1,HAVANA,exon,NaN,3207317.0,.,-,.,"gene_id ""ENSMUSG00000051951.5""; transcript_id ..."
4,chr1,HAVANA,gene,3680155.0,3681788.0,.,+,.,"gene_id ""ENSMUSG00000102348.1""; gene_name ""Gm1..."
